# Fuzzy ARTMAP com dados embaralhados

Esse modelo é formado por duas módulos de FuzzyART.

O aprendizado feito é supervisionado.

Aqui, embaralharemos os dados e avaliaremos o modelo usando a métrica ACC.

<font color="red">Dúvida:</font>
É possível usar a average accuracy mesmo com os dados embaralhados?

## Importando bibliotecas

In [4]:
from torchvision import datasets, transforms
from sklearn.utils import shuffle
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from artlib import FuzzyART
from sklearn.metrics import accuracy_score
from sklearn.metrics.cluster import adjusted_rand_score
import pandas as pd

## Baixando base de dados

In [5]:
transform = transforms.ToTensor()
dataset = datasets.USPS(root='./data', train=True, download=True, transform=transform)

X = np.stack([img.numpy() for img, _ in dataset])
y = np.array([label for _, label in dataset])

X, y = shuffle(X, y, random_state=42)

print(X.shape, y.shape)

(7291, 1, 16, 16) (7291,)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

## Treinando o modelo

In [ ]:
alphas = [1e-6, 1e-3, 1e-2, 1e-1, 1.0, 10.0] 
rhos   = [0.1, 0.3, 0.5, 0.7, 0.9]
betas  = [1.0, 0.5, 0.1]

results = []

for alpha in alphas:
    for rho in rhos:
        for beta in betas:

            model = FuzzyART(rho=rho, alpha=alpha, beta=beta)
            
            lower_bound, upper_bound = model.find_data_bounds(X)
            model.set_data_bounds(lower_bound, upper_bound)

            # Preparar os dados (normalizados internamente pelo modelo)
            train_X_prep = model.prepare_data(X_train)
            test_X_prep  = model.prepare_data(X_test)

            # Treinar
            model.fit(train_X_prep, y_train)

            # Prever
            predictions = model.predict(test_X_prep)

            # Calcular accuracy
            acc = accuracy_score(y_test, predictions)

            results.append({
                'alpha': alpha,
                'rho': rho,
                'beta': beta,
                'accuracy': acc
            })

            #print(f"alpha={alpha:.1e}, rho={rho}, beta={beta} → acc={acc:.3f}")

df_results = pd.DataFrame(results)
best_results = df_results.sort_values(by='accuracy', ascending=False).head(10)


In [8]:
print("10 melhores combinações:")
print(best_results)

10 melhores combinações:
       alpha  rho  beta  accuracy
42  0.010000  0.9   1.0  0.023564
57  0.100000  0.9   1.0  0.000032
0   0.000001  0.1   1.0  0.000000
65  1.000000  0.3   0.1  0.000000
64  1.000000  0.3   0.5  0.000000
63  1.000000  0.3   1.0  0.000000
62  1.000000  0.1   0.1  0.000000
61  1.000000  0.1   0.5  0.000000
60  1.000000  0.1   1.0  0.000000
59  0.100000  0.9   0.1  0.000000
